# START OF PART1

In [17]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

In [18]:
tables_CanadaM = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
print(f'Total tables: {len(tables_CanadaM)}')
df = tables_CanadaM[0]
df.head()

Total tables: 3


,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [19]:
df = df[df['Borough'] != 'Not assigned']
df_grp = df.groupby('Postal Code').count()
df_grp[df_grp['Borough'] >1 ]
df[df['Neighbourhood'] == 'Not assigned']

,Postal Code,Borough,Neighbourhood


In [20]:
df.shape

(103, 3)

# END OF PART1

# START OF PART2

In [21]:
geo_cordinates = pd.read_csv('http://cocl.us/Geospatial_data')
geo_cordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [23]:
df12merged = pd.merge(df,geo_cordinates, how='left',  on=['Postal Code'])
df12merged.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [24]:
df12merged.shape

(103, 5)

# END OF PART2

# START OF PART3

In [32]:
address = 'Goodwood Park Cres East York, Toronto'
geolocator = Nominatim(user_agent="Narendra_Tomar")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Geograpical coordinates of Toronto are as follows {}, {}.'.format(latitude, longitude))
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lng, label in zip(df12merged['Latitude'], df12merged['Longitude'], df12merged['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)    
map_toronto

Geograpical coordinates of Toronto are as folows 43.6948316, -79.2989997.


In [33]:
CLIENT_ID = 'your-client-ID'
CLIENT_SECRET = 'your-client-secret'
CLIENT_ID = 'CJW33UYYHFE03JRASXFE1HH5APKYUGNX0FQPUWKACFRNJ4BN'
CLIENT_SECRET = 'PHAE3SK4ZFIDOBOWPQ204YKBR4L3DUZTA0KOW5XGTV3ON13P'
VERSION = '20180605' # Foursquare API version
LIMIT = 100

In [35]:
df12merged.loc[0, 'Neighbourhood']

'Parkwoods'

In [39]:
neighborhoodlatitude = df12merged.loc[0, 'Latitude']
neighborhoodlongitude = df12merged.loc[0, 'Longitude']
neighborhoodname = df12merged.loc[0, 'Neighbourhood']
print('The latitude and longitude values of {} are as follows ---{}, {}.'.format(neighborhood_name, neighborhoodlatitude, neighborhoodlongitude))

The latitude and longitude values of Parkwoods are as follows ---43.7532586, -79.3296565.


In [40]:
LIMIT = 100
radius = 500
VERSION = '20180605'
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, radius, LIMIT)
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '602999539671424fd325eb39'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 2,
  'suggestedBounds': {'ne': {'lat': 43.757758604500005,
    'lng': -79.32343823984928},
   'sw': {'lat': 43.7487585955, 'lng': -79.33587476015072}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
       'name': 'Brookbanks Park',
       'location': {'address': 'Toronto',
        'lat': 43.751976046055574,
        'lng': -79.33214044722958,
        'labeledLatLngs': 

In [41]:
def categorytype(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [43]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues['venue.categories'] = nearby_venues.apply(categorytype, axis=1)
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  from ipykernel import kernelapp as app


,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,Variety Store,Food & Drink Shop,43.751974,-79.333114


# END OF PART3